<h1 align= "center"> <strong> Pipeline Configurations </strong> </h1>
<center>---------------------------- </center>

# Table of Contents
* [1. Writing JSON to a file in python](#1.-Writing-JSON-to-a-file-in-python)
* [2. Reading JSON from a file using python](#2.-Reading-JSON-from-a-file-using-python)

## 1. Writing JSON to a file in python

In [10]:
project_name = "awsworkshop"
job_name='inference'
team = "team2"
BUName = "awsworkshop"
user_name = "mlops"

In [18]:
import json
  
# Data to be written
dictionary ={
    "pipeline": {
        "pipeline_config" : {
            "model_package_group_name": "AWSWorkShop-ModelPackageGroup",
            "pipeline_name": f"{BUName}-{team}-{job_name}-pipeline-",
            "base_job_prefix": f"{BUName}-{team}-{job_name}",
            "env": "dev",
            "project_name": f"{project_name}"
        },
       "inputs": {
            "input1": "s3://dlk-cloud-tier-10-preprocessed-ml-dev/aws_workshop/inference_input_dataset/telco_cutomer_inference.csv",
            "input2": "enter the second s3 input file location.csv"
        },
        "generic_tags": {
            "ProjectName": "aws_workshop",
            "DepartmentName": "dialog_ml",
            "UsecaseName": f"{team}",
            "ResourceName": "sagemaker_notebook",
            "OwnerName": "saranga_gunasekara",
            "BUName": f"{BUName}"
        }
    },
    "preprocessing": {
        "estimator_config": {
            "ecr_repository": f"{project_name}-preprocessing-image",
            "instance_count": 1,
            "instance_type": "ml.m5.large"
        },
        "inputs": {
            "code" : f"awsworkshop_{team}_inf_preprocessing/preprocessing.py"
        },
    },
    "create_model": {
        "estimator_config": {
            "job_name": f"{project_name}{team}-model",
            "instance_count": 2,
            "instance_type": "ml.m5.12xlarge",
        },
    },
    "inference": {
        "estimator_config": {
            "ecr_repository": f"{project_name}_{team}_{user_name}-inference-image",
            "instance_count": 2,
            "instance_type": "ml.m5.12xlarge",
            "content_type": "text/csv"
        },
    },
    "postpreprocessing": {
        "inputs": {
            "code" : f'awsworkshop_{team}_inf_postprocessing/postprocessing.py'
        },

}
}

In [19]:
import json
import boto3    
s3 = boto3.resource('s3')
s3object = s3.Object('dlk-cloud-tier-8-code-ml-dev', f'config_files/pipeline_config/{project_name}/{job_name}/{user_name}/pipeline_config.json')

s3object.put(
    Body=(bytes(json.dumps(dictionary).encode('UTF-8')))
)

{'ResponseMetadata': {'RequestId': 'QR2JK1BMSFXP8VEG',
  'HostId': '9is19HKrgY27is54fNGQRC9Q8hcRZl+z6cqsnx0vLn1yP/XNLl2rYU/gSOTDoV7wBLNLDcrO/Po=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '9is19HKrgY27is54fNGQRC9Q8hcRZl+z6cqsnx0vLn1yP/XNLl2rYU/gSOTDoV7wBLNLDcrO/Po=',
   'x-amz-request-id': 'QR2JK1BMSFXP8VEG',
   'date': 'Wed, 02 Mar 2022 04:01:43 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"91aa5de3919e800c14ef9c478739b859"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"91aa5de3919e800c14ef9c478739b859"',
 'ServerSideEncryption': 'AES256'}

## 2. Reading JSON from a file using python

In [13]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

In [14]:
env = 'dev'

In [15]:
import json
import boto3

def getJsonData(bucket_name,key_name):
    print("[LOG]", bucket_name,'---------')
    print("[LOG]", key_name,'--------------')
      
    s3_client = boto3.client('s3')
    csv_obj = s3_client.get_object(Bucket=bucket_name, Key=key_name)
    
    body = csv_obj['Body']
    
    json_string = body.read().decode('utf-8')
    json_content = json.loads(json_string)
    
    return json_content

In [16]:
config_bucket = ParameterString(
        name="ConfigBucket",
        default_value=f"dlk-cloud-tier-8-code-ml-{env}",  # Change this to point to the s3 location of your raw input data.
    )
config_s3_prefix = ParameterString(
        name="ConfigS3Prefix",
        default_value=f'config_files/pipeline_config/{project_name}/{job_name}/{user_name}/pipeline_config.json',  # Change this to point to the s3 location of your raw input data.
    )    
    
config = getJsonData(config_bucket,config_s3_prefix)

[LOG] dlk-cloud-tier-8-code-ml-dev ---------
[LOG] config_files/pipeline_config/awsworkshop/inference/mlops/pipeline_config.json --------------


In [17]:
config["postpreprocessing"]['inputs']['code']

'awsworkshop_team2_inf_postprocessing/postprocessing.py'

In [25]:
config 

{'pipeline': {'pipeline_config': {'model_package_group_name': 'AWSWorkShop-ModelPackageGroup',
   'pipeline_name': 'awsworkshop-team1-pipeline-',
   'base_job_prefix': 'awsworkshop-team1-inference',
   'env': 'dev',
   'project_name': 'awsworkshop'},
  'inputs': {'input1': 's3://dlk-cloud-tier-10-preprocessed-ml-dev/aws_workshop/inference_input_dataset/telco_cutomer_inference.csv',
   'input2': 'enter the second s3 input file location.csv'},
  'generic_tags': {'ProjectName': 'aws_workshop',
   'DepartmentName': 'dialog_ml',
   'UsecaseName': 'team1',
   'ResourceName': 'sagemaker_notebook',
   'OwnerName': 'saranga_gunasekara',
   'BUName': 'awsworkshop'}},
 'preprocessing': {'estimator_config': {'ecr_repository': 'awsworkshop-preprocessing-image',
   'instance_count': 1,
   'instance_type': 'ml.m5.large'},
  'inputs': {'code': 'preprocessing_inference/preprocessing_config_inf.py'}},
 'create_model': {'estimator_config': {'job_name': 'awsworkshopteam1-model',
   'instance_count': 2,
  

In [12]:
config['pipeline']['pipeline_config']['model_package_group_name']

'SalesStockPred-ModelPackageGroup'

In [13]:
config['pipeline']['pipeline_config']['pipeline_name']

'SalesStockPred-pipeline-'

In [14]:
config['pipeline']['pipeline_config']['base_job_prefix']

'SalesStockPred'

In [15]:
config['pipeline']['pipeline_config']['env']

'dev'

In [16]:
config['pipeline']['pipeline_config']['project_name']

'sales_stockpred'

In [18]:
config['pipeline']['inputs']['input1']

's3://dlk-cloud-tier-10-preprocessed-ml-dev/sales_stockpred/temp_data/gc_retailers_3yr/temp/'

In [19]:
config['pipeline']['inputs']['input2']

's3://dlk-cloud-tier-10-preprocessed-ml-dev/sales_stockpred/temp_data/gc_retailers_3yr/gc_retailers_2022_JAN10.csv'

In [24]:
config['pipeline']['generic_tags']['UsecaseName']

'sales-stockpred'

In [28]:
config['preprocessing']['estimator_config']

{'ecr_repository': 'sales-stockpred-preprocessing-image',
 'instance_count': 1,
 'instance_type': 'ml.m5.large'}

In [33]:
config['train_model']['estimator_config']['instance_count']

2